# Prune the AR6 database climate dump

All we want is the 1202 vetted scenarios from FaIR v1.6.2 with
- total anthro forcing (50th percentile)
- CO2 forcing (50th percentile)
- CH4 forcing (50th percentile)
- N2O forcing (50th percentile)

In [1]:
import pandas as pd
from tqdm.auto import tqdm

/Users/chris/miniconda3/envs/frida-forcing/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ar6_df = pd.read_csv('../scenario_data/1668008030411-AR6_Scenarios_Database_World_ALL_CLIMATE_v1/AR6_Scenarios_Database_World_ALL_CLIMATE_v1.1.csv')

In [3]:
ar6_df

,Model,Scenario,Region,Variable,Unit,1995,1996,1997,1998,1999,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
0,AIM/CGE 2.0,ADVANCE_2020_1.5C-2100,World,AR6 climate diagnostics|Atmospheric Concentrat...,ppb,1755.229625,1757.190593,1761.501078,1770.29068,1778.199514,...,997.94491,999.275554,1001.411412,1004.270396,1007.778784,1011.400424,1014.653825,1017.576527,1020.202244,1022.561253
1,AIM/CGE 2.0,ADVANCE_2020_1.5C-2100,World,AR6 climate diagnostics|Atmospheric Concentrat...,ppb,1755.229625,1757.190593,1761.501078,1770.29068,1778.199514,...,997.94491,999.275554,1001.411412,1004.270396,1007.778784,1011.400424,1014.653825,1017.576527,1020.202244,1022.561253
2,AIM/CGE 2.0,ADVANCE_2020_1.5C-2100,World,AR6 climate diagnostics|Atmospheric Concentrat...,ppb,1755.229625,1757.190593,1761.501078,1770.29068,1778.199514,...,997.94491,999.275554,1001.411412,1004.270396,1007.778784,1011.400424,1014.653825,1017.576527,1020.202244,1022.561253
3,AIM/CGE 2.0,ADVANCE_2020_1.5C-2100,World,AR6 climate diagnostics|Atmospheric Concentrat...,ppb,1755.229625,1757.190593,1761.501078,1770.29068,1778.199514,...,997.94491,999.275554,1001.411412,1004.270396,1007.778784,1011.400424,1014.653825,1017.576527,1020.202244,1022.561253
4,AIM/CGE 2.0,ADVANCE_2020_1.5C-2100,World,AR6 climate diagnostics|Atmospheric Concentrat...,ppb,1755.229625,1757.190593,1761.501078,1770.29068,1778.199514,...,997.94491,999.275554,1001.411412,1004.270396,1007.778784,1011.400424,1014.653825,1017.576527,1020.202244,1022.561253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024710,WITCH-GLOBIOM 4.4,CD-LINKS_NoPolicy,World,Secondary Energy|Electricity|Renewables (incl....,EJ/yr,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,194.113407
2024711,WITCH-GLOBIOM 4.4,CD-LINKS_NoPolicy,World,Secondary Energy|Electricity|Solar,EJ/yr,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.982722
2024712,WITCH-GLOBIOM 4.4,CD-LINKS_NoPolicy,World,Secondary Energy|Electricity|Wind,EJ/yr,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113.919560
2024713,WITCH-GLOBIOM 4.4,CD-LINKS_NoPolicy,World,Secondary Energy|Gases,EJ/yr,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,207.795108


In [4]:
xl = pd.ExcelFile('../scenario_data/1668008030411-AR6_Scenarios_Database_World_ALL_CLIMATE_v1/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx')
xl.sheet_names

['README',
 'meta_Ch3vetted_withclimate',
 'meta',
 'categories_indicators_doc',
 'Illustrative_Pathways']

In [5]:
meta_df = xl.parse('meta')

In [6]:
wanted_vars = [
    'AR6 climate diagnostics|Effective Radiative Forcing|Basket|Anthropogenic|FaIRv1.6.2|50.0th Percentile',
    'AR6 climate diagnostics|Effective Radiative Forcing|CO2|FaIRv1.6.2|50.0th Percentile',
    'AR6 climate diagnostics|Effective Radiative Forcing|CH4|FaIRv1.6.2|50.0th Percentile',
    'AR6 climate diagnostics|Effective Radiative Forcing|N2O|FaIRv1.6.2|50.0th Percentile',
    'AR6 climate diagnostics|Harmonized|Emissions|CO2|Energy and Industrial Processes',
    'AR6 climate diagnostics|Effective Radiative Forcing|FaIRv1.6.2|50.0th Percentile',
]

In [7]:
new_df = pd.DataFrame(columns = ar6_df.columns)
new_df

,Model,Scenario,Region,Variable,Unit,1995,1996,1997,1998,1999,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100


In [8]:
for ms, _ in tqdm(meta_df[meta_df['Category'].isin([f'C{i}' for i in range(1, 9)])].groupby(["Model", "Scenario"])):
    model = ms[0]
    scenario = ms[1]
    #ar6_df.loc[(ar6_df["Model"]==model)&(ar6_df["Scenario"]==scenario)&(ar6_df["Variable"].isin(wanted_vars))]
    new_df = pd.concat(
        [
            new_df,
            ar6_df.loc[
                (ar6_df["Model"]==model)&(ar6_df["Scenario"]==scenario)&(ar6_df["Variable"].isin(wanted_vars))
            ]
        ]
    )

100%|███████████████████████████████████████| 1202/1202 [08:01<00:00,  2.49it/s]


In [9]:
#list(ar6_df.Variable.unique())

In [10]:
new_df

,Model,Scenario,Region,Variable,Unit,1995,1996,1997,1998,1999,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
63087,AIM/CGE 2.0,SSP1-26,World,AR6 climate diagnostics|Effective Radiative Fo...,W/m^2,1.636460,1.684510,1.627874,1.671092,1.815591,...,2.981693,2.976049,2.970522,2.964628,2.959076,2.953401,2.948116,2.942070,2.936613,2.931564
63249,AIM/CGE 2.0,SSP1-26,World,AR6 climate diagnostics|Effective Radiative Fo...,W/m^2,0.503463,0.504183,0.505825,0.509215,0.512248,...,0.167832,0.166360,0.164919,0.163506,0.162117,0.160750,0.159403,0.158074,0.156761,0.155461
63267,AIM/CGE 2.0,SSP1-26,World,AR6 climate diagnostics|Effective Radiative Fo...,W/m^2,1.471549,1.496910,1.522611,1.551692,1.580700,...,2.585070,2.579409,2.573590,2.567569,2.561949,2.556296,2.550965,2.545383,2.540125,2.534307
63303,AIM/CGE 2.0,SSP1-26,World,AR6 climate diagnostics|Effective Radiative Fo...,W/m^2,1.726261,1.829280,1.810633,1.908043,2.083303,...,3.015661,3.036785,3.019582,2.991755,2.968416,2.942514,2.931088,2.919937,2.924517,2.940808
63501,AIM/CGE 2.0,SSP1-26,World,AR6 climate diagnostics|Effective Radiative Fo...,W/m^2,0.128452,0.131823,0.134733,0.137085,0.139695,...,0.289868,0.290365,0.290846,0.291311,0.291761,0.292194,0.292613,0.293016,0.293404,0.293778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024128,WITCH-GLOBIOM 4.4,CD-LINKS_NoPolicy,World,AR6 climate diagnostics|Effective Radiative Fo...,W/m^2,0.503463,0.504183,0.505825,0.509215,0.512248,...,0.641046,0.639387,0.637588,0.635664,0.633625,0.631484,0.629250,0.626931,0.624535,0.622070
2024146,WITCH-GLOBIOM 4.4,CD-LINKS_NoPolicy,World,AR6 climate diagnostics|Effective Radiative Fo...,W/m^2,1.471549,1.496910,1.522611,1.551692,1.580700,...,6.072179,6.128475,6.184595,6.240336,6.295445,6.348240,6.401096,6.453596,6.505259,6.556166
2024182,WITCH-GLOBIOM 4.4,CD-LINKS_NoPolicy,World,AR6 climate diagnostics|Effective Radiative Fo...,W/m^2,1.726261,1.829280,1.810633,1.908043,2.083303,...,7.221970,7.308284,7.354097,7.388602,7.424990,7.449062,7.497821,7.548879,7.604435,7.694667
2024380,WITCH-GLOBIOM 4.4,CD-LINKS_NoPolicy,World,AR6 climate diagnostics|Effective Radiative Fo...,W/m^2,0.128452,0.131823,0.134733,0.137085,0.139695,...,0.366853,0.368540,0.370207,0.371835,0.373453,0.375060,0.376664,0.378249,0.379797,0.381309


In [20]:
othererf_df = pd.DataFrame(columns = ar6_df.columns)

In [21]:
for idx, (ms, _) in tqdm(enumerate(new_df.groupby(["Model", "Scenario"]))):
    model = ms[0]
    scenario = ms[1]
    othererf_df.loc[idx, 'Model'] = model
    othererf_df.loc[idx, 'Scenario'] = scenario
    othererf_df.loc[idx, 'Variable'] = 'AR6 climate diagnostics|Effective Radiative Forcing|Basket|Anthropogenic non-CO2-CH4-N2O|FaIRv1.6.2|50.0th Percentile'
    othererf_df.loc[idx, 'Region'] = 'World'
    othererf_df.loc[idx, 'Unit'] = 'W/m^2'
    othererf_df.loc[idx, '1995':] = (
        new_df.loc[
            (new_df["Model"]==model)&
            (new_df["Scenario"]==scenario)&
            (new_df["Variable"]=='AR6 climate diagnostics|Effective Radiative Forcing|Basket|Anthropogenic|FaIRv1.6.2|50.0th Percentile'),
        '1995':].values-new_df.loc[
            (new_df["Model"]==model)&
            (new_df["Scenario"]==scenario)&
            (new_df["Variable"]=='AR6 climate diagnostics|Effective Radiative Forcing|CO2|FaIRv1.6.2|50.0th Percentile'),
        '1995':].values-new_df.loc[
            (new_df["Model"]==model)&
            (new_df["Scenario"]==scenario)&
            (new_df["Variable"]=='AR6 climate diagnostics|Effective Radiative Forcing|CH4|FaIRv1.6.2|50.0th Percentile'),
        '1995':].values-new_df.loc[
            (new_df["Model"]==model)&
            (new_df["Scenario"]==scenario)&
            (new_df["Variable"]=='AR6 climate diagnostics|Effective Radiative Forcing|N2O|FaIRv1.6.2|50.0th Percentile'),
        '1995':].values
    )
#     new_df = pd.concat(
#         [
#             new_df,
#             ar6_df.loc[
#                 (ar6_df["Model"]==model)&(ar6_df["Scenario"]==scenario)&(ar6_df["Variable"].isin(wanted_vars))
#             ]
#         ]

1202it [00:16, 74.35it/s]


In [23]:
new_df = pd.concat((othererf_df, new_df))

In [26]:
new_df.sort_values(by=['Model', 'Scenario']).to_csv('../data/ar6_wg3_scenarios.csv', index=False)